In [1]:
from bs4 import BeautifulSoup as BSHTML
import pandas as pd
from urllib.request import urlopen

In [2]:
df = pd.DataFrame()
data_dict = {}

author_urls = {
    'Chimamanda-Ngozi-Adichie': "Chimamanda-Ngozi-Adichie/e/B00PODW5UG",
    # 'Elise Bohan': "https://www.amazon.com/Elise-Bohan/e/B09X6CPCYR"
}

# Get the HTML source

author_url_prefix = 'https://www.amazon.com/'
author = 'Chimamanda-Ngozi-Adichie'
page = urlopen(author_url_prefix + author_urls[author])
soup = BSHTML(page)

In [3]:

data_dict[author] = {}

# This dictionary structure - 
# key - the name of the author
# value is a dictionary again 
# dictionary key - name of the recommended author, img_url, amazon page url

In [4]:
# Get the list of authors, and their image URLs
# from the soup. 

rec_authors = []
auth_img = []
for item in soup.find_all(class_="authorListImage"):
    rec_auth = item["alt"]
    rec_authors.append(rec_auth)
    
    data_dict[author][rec_auth] = []
    data_dict[author][rec_auth].append(item["src"])


# data_dict

In [5]:
# The soups gives a lot of links; need to collect only the ones with author names
# in certain format. Get that format first.
auth_links = []
auth_link_names = {}
for auth in rec_authors:
    auth_link_names[auth] = ("-".join(n.strip().replace('.', '') for n in auth.split()))

for item in soup.findAll(class_="a-link-normal"):
    if '/e/' in item["href"]:
        link = item["href"]
        idx = link.find('?ref=')
        if idx > 0:
            link = link[:idx]
        auth_links.append(link)

auth_links = list(set(auth_links))

for rec_auth, link in auth_link_names.items():
    for auth_link in auth_links:
        if link in auth_link:
            data_dict[author][rec_auth].append(auth_link)
            author_urls[rec_auth] = auth_link

# This will ensure that for all the recommended authors, their links are found
# assert(len(auth_link_names) == 0)



,author,recommended_author,img_url,author_url
0,Chimamanda-Ngozi-Adichie,Bethel Joshua,https://m.media-amazon.com/images/S/amzn-autho...,/Bethel-Joshua/e/B0B9FP3WCG
1,Chimamanda-Ngozi-Adichie,Jhumpa Lahiri,https://m.media-amazon.com/images/I/61-3-dPgGf...,/Jhumpa-Lahiri/e/B001H6GTG0
2,Chimamanda-Ngozi-Adichie,Barbara A. Mowat,https://m.media-amazon.com/images/I/51z8ocAA75...,/Barbara-A-Mowat/e/B001IOH98I
3,Chimamanda-Ngozi-Adichie,Marjane Satrapi,https://m.media-amazon.com/images/I/51z6DF+kRp...,/Marjane-Satrapi/e/B001IGFN9I
4,Chimamanda-Ngozi-Adichie,Khaled Hosseini,https://m.media-amazon.com/images/I/31M4JeJpTg...,/Khaled-Hosseini/e/B001HCX9N0
5,Chimamanda-Ngozi-Adichie,Yaa Gyasi,https://m.media-amazon.com/images/I/41aFsWMHGm...,/Yaa-Gyasi/e/B01MZ40HXZ
6,Chimamanda-Ngozi-Adichie,Louise Erdrich,https://m.media-amazon.com/images/I/61BwJXBudx...,/Louise-Erdrich/e/B000APV798
7,Chimamanda-Ngozi-Adichie,Thomas C. Foster,https://m.media-amazon.com/images/I/310b9Q5+lg...,/Thomas-C-Foster/e/B001H6PR3Q
8,Chimamanda-Ngozi-Adichie,Toni Morrison,https://m.media-amazon.com/images/I/31DmLlzXyM...,/Toni-Morrison/e/B000APT7NQ
9,Chimamanda-Ngozi-Adichie,Homer,https://m.media-amazon.com/images/S/amzn-autho...,/Homer/e/B000AP7OK4


In [58]:
for item in soup.find('span', {'id': 'author_biography'}):
    print (item.text.strip())


CHIMAMANDA NGOZI ADICHIE's work has appeared in numerous publications, including The New Yorker and Granta. She is the author of the novels Purple Hibiscus; Half of a Yellow Sun, which won the Orange Prize; Americanah, which won the NBCC Award and was a New York Times, Washington Post, Chicago Tribune, and Entertainment Weekly Best Book of the Year; the story collection The Thing Around Your Neck; and the essay We Should All Be Feminists. A recipient of a MacArthur Fellowship, she divides her time between the United States and Nigeria.



In [62]:
df = pd.DataFrame.from_dict(data_dict[author], orient='index').reset_index()

In [63]:
df.columns = ['recommended_author', 'img_url', 'author_url']
df.loc[:, 'author'] = author
columns = ['author', 'recommended_author', 'img_url', 'author_url'] 
df[columns]

,author,recommended_author,img_url,author_url
0,Chimamanda-Ngozi-Adichie,Bethel Joshua,https://m.media-amazon.com/images/S/amzn-autho...,/Bethel-Joshua/e/B0B9FP3WCG
1,Chimamanda-Ngozi-Adichie,Jhumpa Lahiri,https://m.media-amazon.com/images/I/61-3-dPgGf...,/Jhumpa-Lahiri/e/B001H6GTG0
2,Chimamanda-Ngozi-Adichie,Barbara A. Mowat,https://m.media-amazon.com/images/I/51z8ocAA75...,/Barbara-A-Mowat/e/B001IOH98I
3,Chimamanda-Ngozi-Adichie,Marjane Satrapi,https://m.media-amazon.com/images/I/51z6DF+kRp...,/Marjane-Satrapi/e/B001IGFN9I
4,Chimamanda-Ngozi-Adichie,Khaled Hosseini,https://m.media-amazon.com/images/I/31M4JeJpTg...,/Khaled-Hosseini/e/B001HCX9N0
5,Chimamanda-Ngozi-Adichie,Yaa Gyasi,https://m.media-amazon.com/images/I/41aFsWMHGm...,/Yaa-Gyasi/e/B01MZ40HXZ
6,Chimamanda-Ngozi-Adichie,Louise Erdrich,https://m.media-amazon.com/images/I/61BwJXBudx...,/Louise-Erdrich/e/B000APV798
7,Chimamanda-Ngozi-Adichie,Thomas C. Foster,https://m.media-amazon.com/images/I/310b9Q5+lg...,/Thomas-C-Foster/e/B001H6PR3Q
8,Chimamanda-Ngozi-Adichie,Toni Morrison,https://m.media-amazon.com/images/I/31DmLlzXyM...,/Toni-Morrison/e/B000APT7NQ
9,Chimamanda-Ngozi-Adichie,Homer,https://m.media-amazon.com/images/S/amzn-autho...,/Homer/e/B000AP7OK4
